<a href="https://colab.research.google.com/github/aborundiya/CNN/blob/master/CIFAR_InceptionNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import copy 
import torch 
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn 
import torch.optim as optim 
from torchvision import models
import matplotlib.pyplot as plt 

#Check for GPU 

In [6]:
device = torch.device('cuda:0'if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [0]:
transform_train = transforms.Compose([
                                      transforms.RandomResizedCrop(299),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

transform_test = transforms.Compose([
                                     transforms.RandomResizedCrop(299),
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [8]:
train_set = torchvision.datasets.CIFAR10('./data',train=True,download=True,transform=transform_train)
test_set = torchvision.datasets.CIFAR10('./data',train=False,download=True,transform=transform_test)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [0]:
batch_size = 16 
num_classes = 10
trainloader = torch.utils.data.DataLoader(train_set,batch_size=batch_size,shuffle=True)
testloader = torch.utils.data.DataLoader(test_set,batch_size=batch_size,shuffle=False)


In [10]:
incepnet = models.inception_v3(pretrained=True)

Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /root/.cache/torch/checkpoints/inception_v3_google-1a9a5a14.pth


In [0]:
for param in incepnet.parameters():
  param.requires_grad = False 

In [12]:
print(incepnet)

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

# Altering the aux and Fc layer to suit the CIFAR dataset

In [0]:
aux_final_in_features = incepnet.AuxLogits.fc.in_features
incepnet.AuxLogits.fc = nn.Linear(aux_final_in_features,num_classes)

In [14]:
for param in incepnet.parameters():
  if param.requires_grad:
    print(param.shape)

torch.Size([10, 768])
torch.Size([10])


In [0]:
final_in_features = incepnet.fc.in_features
incepnet.fc = nn.Linear(final_in_features,num_classes)

In [16]:
for param in incepnet.parameters():
  if param.requires_grad:
    print(param.shape)

torch.Size([10, 768])
torch.Size([10])
torch.Size([10, 2048])
torch.Size([10])


#Training and evaluation of the network

In [0]:
inception = incepnet.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.SGD(incepnet.parameters(), lr = 0.01)

In [0]:
def evalaution_inception(dataloader, model):
  total , correct = 0 , 0 
  for data in dataloader: 
    inputs, labels = data 
    inputs, labels = inputs.to(device) , labels.to(device)
    outputs , aux_outputs = model(inputs)
    _ , pred = torch.max(outputs.data , 1 )
    total += labels.size(0)
    correct += (pred == labels).sum().item()
  return 100 * (correct / total )

In [19]:

loss_epoch_arr = []
max_epochs = 1 
min_loss = 1000
n_iters = np.ceil(50000/batch_size)

for epoch in range(max_epochs):

  for i , data in enumerate(trainloader,0):

    inputs, labels = data 
    inputs, labels = inputs.to(device), labels.to(device)
    opt.zero_grad()
    outputs , aux_outputs = incepnet(inputs)
    loss = loss_fn(outputs, labels) + 0.3 * loss_fn(aux_outputs, labels)  # thus to accomodate the aux loss 
    loss.backward()
    opt.step()
    if min_loss > loss : 
      min_loss = loss 
      best_model = copy.deepcopy(incepnet.state_dict())
      print("Min loss %0.2f" % min_loss) 

    if i % 100 == 0 : 
      print("iteration: %d / %d , Loss : %0.2f"% (i , n_iters, loss.item()))

    del inputs, labels, outputs, aux_outputs 
    torch.cuda.empty_cache()

  loss_epoch_arr.append(loss.item())
  print("Epoch %d/%d, Test Accuracy : %0.2f , Train Accuracy : %0.2f"% (epoch , max_epochs,
                                                                        evalaution_inception(testloader, incepnet),
                                                                        evalaution_inception(trainloader,incepnet))) 

Min loss 3.12
iteration: 0 / 3125 , Loss : 3.12
Min loss 3.02
Min loss 2.94
Min loss 2.92
Min loss 2.86
Min loss 2.84
Min loss 2.80
Min loss 2.74
Min loss 2.64
Min loss 2.48
iteration: 100 / 3125 , Loss : 2.61
Min loss 2.42
Min loss 2.41
Min loss 2.24
Min loss 2.20
Min loss 2.16
iteration: 200 / 3125 , Loss : 2.52
Min loss 2.16
Min loss 2.10
Min loss 2.06
Min loss 2.00
iteration: 300 / 3125 , Loss : 2.28
Min loss 1.96
Min loss 1.83
Min loss 1.82
iteration: 400 / 3125 , Loss : 2.59
iteration: 500 / 3125 , Loss : 2.48
Min loss 1.77
Min loss 1.70
iteration: 600 / 3125 , Loss : 2.20
Min loss 1.63
Min loss 1.52
iteration: 700 / 3125 , Loss : 2.05
Min loss 1.50
iteration: 800 / 3125 , Loss : 1.60
iteration: 900 / 3125 , Loss : 2.27
Min loss 1.41
iteration: 1000 / 3125 , Loss : 2.38
iteration: 1100 / 3125 , Loss : 1.72
iteration: 1200 / 3125 , Loss : 1.91
iteration: 1300 / 3125 , Loss : 1.70
Min loss 1.38
iteration: 1400 / 3125 , Loss : 2.45
Min loss 1.37
Min loss 1.26
iteration: 1500 / 3125 

In [21]:
incepnet.load_state_dict(best_model)
print(evalaution_inception(trainloader, inception), evalaution_inception(testloader, inception))

46.062 46.129999999999995
